<a href="https://colab.research.google.com/github/Fazkek/OpenWeatherAPI/blob/main/1_%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D1%81_%D0%91%D0%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подключение к базам данных

SQLite - самая простая база данных, к которой можно подключиться с помощью Python. По умолчанию стандартная библиотека Python уже содержит модуль sqlite3 и для работы не требуется устанавливать какие-либо внешние модули. Кроме того, SQLite база данных не требует сервера. Она просто читает и записывает данные в файл. Подключимся с помощью sqlite3 к базе данных:

In [1]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

Код работает следующим образом:

Строки 1 и 2 – импорт sqlite3 и класса Error.
Строка 4 определяет функцию create_connection(), которая принимает путь к базе данных SQLite.
Строка 7 использует метод connect() и принимает в качестве параметра путь к базе данных SQLite. Если база данных в указанном месте существует, будет установлено соединение. В противном случае по указанному пути будет создана новая база данных и так же установлено соединение.
В строке 8 выводится состояние успешного подключения к базе данных.
Строка 9 перехватывает любое исключение, которое может быть получено, если методу .connect() не удастся установить соединение.
В строке 10 отображается сообщение об ошибке в консоли.
sqlite3.connect(path) возвращает объект connection. Этот объект может использоваться для выполнения запросов к базе данных SQLite. Следующий скрипт формирует соединение с базой данных SQLite:

In [2]:
path = "MasterMargarita.sqlite"
connection = create_connection(path)

Connection to SQLite DB successful


Выполнив вышеуказанный скрипт, вы увидите, как в указанной папке появится файл базы данных MasterMargarita.sqlite. 

# Создание таблиц

В данном разделе изучается формирование таблицы внутри базы данных. Нужно получить и связать четыре таблицы: users, fragments, keywords, emotions.

Таблица users будет содержать данные об асессорах, которые занимаются разметкой книги.
Таблица fragments будет содержить фрагменты книги, размеченные асессорами, а также дополнительную информацию для данных фрагментов. Например, если фрагмены искались для конкретного персонажа или связанного ключевого слова.
Таблица keywords будет содержать инфомацию о ключевых словах - список их синонимов и эмоции, с ними связанные.
Таблица emotions будет содержать конкретные эмоции, связывающие асессора и размеченные им фрагменты.

Для работы с базами данных - создания и заполнения таблиц, выполнения запросов, управления обновлением и удалением строк - в SQLite используется метод cursor.execute(). В этом разделе мы определим функцию execute_query(), которая использует этот метод. Функция будет принимать объект connection и строку запроса. Далее строка запроса будет передаваться методу execute( ). В этом разделе он будет использоваться для формирования таблиц, а в следующих – мы применим его для выполнения запросов на обновление и удаление.

Все, что нужно сделать, это сохранить запрос в строковой переменной и затем передать эту переменную cursor.execute().

Итак, начнем с определения функции execute_query():

In [3]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

Теперь напишем передаваемый запрос (query):

In [4]:
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  surname TEXT NOT NULL,
  age INTEGER,
  gender TEXT,
  school TEXT,
  class INTEGER
);
"""

В запросе говорится, что нужно создать таблицу users со следующими столбцами: id, name, surname, age, gender, school, class. Чтобы появилась таблица, вызываем execute_query(). 

Можно ли записать школу INTEGER, а класс TEXT?
Добавьте в таблицу ещё два поля - город (обязательное) и год заполнения таблицы (обязательное).

Передаём объект connection, который мы описали в предыдущем разделе, вместе с только что подготовленной строкой запроса create_users_table:

In [5]:
execute_query(connection, create_users_table)

Query executed successfully


Теперь составим таблицу с описанием фрагментов текста. Здесь укажем, из какой главы взят фрагмент (chapter), внесем его в таблицу (content). Также добавим два необязательных поля: герой (hero) и слово (word), если фрагмент специально выбирался для характеристики героя или какого-то слова. Возможно также сочетание героя и слова, связанного с ним.
Следующий запрос используется для создания таблицы fragments:

In [6]:
create_fragments_table = """
CREATE TABLE IF NOT EXISTS fragments(
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  chapter INTEGER NOT NULL, 
  content TEXT NOT NULL,
  hero TEXT, 
  word TEXT,
  user_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id)
);
"""

Поскольку асессор может анализировать несколько фрагментов, между users и fragments имеет место отношение один-ко-многим. Поэтому в таблице появляется ключ user_id, который ссылается на столбец id в таблице users. Выполняем следующий скрипт для построения таблицы fragments:

In [7]:
execute_query(connection, create_fragments_table)

Query executed successfully


Перейдем к таблице ключевых слов keywords. Ключевые слова связаны с определенным фрагментом. Заметим, что слово, относительно которого мы ищем фрагмент, если работаем по схеме разметки "Поиск слова" и ключевое слово, которое мы выделяем в фрагменте - это разные понятия. Поэтому и в базе данных они находятся в разных конструкциях. 

Ключевое слово содержит поля: ключевой идентификатор (id), название ключевого слова (title), возможные синонимы (synonyms), возможную эмоцию (word_emotion), а также поля, связанные с другими таблицами - user_id и fragment_id. Заметим, что выделять ключевые слова во фрагменте может не тот же пользователь, который выделил сам фрагмент. Так ли это в данной таблице?

Формируем следующим скриптом таблицу keywords:

In [8]:
create_keywords_table = """
CREATE TABLE IF NOT EXISTS keywords (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  title TEXT NOT NULL, 
  synonyms TEXT,
  word_emotion TEXT,
  user_id INTEGER NOT NULL, 
  fragment_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) 
  FOREIGN KEY (fragment_id) REFERENCES fragment (id)
);
"""

execute_query(connection, create_keywords_table)

Query executed successfully


Далее перейдем к таблице эмоций. Каждая эмоция определяется пользователем. Эмоции могут иметь синонимы, их желательно перечислить в поле synonyms. Для каждой эмоции необходимо указать, она берется из базовой модели на 8 эмоций (base) или пользовательской (users). Если указана пользовательская модель, то необходимо добавить описание модели, поэтому в качестве задания необходимо модель вынести в отдельную таблицу и подумать, какие поля для нее необходимы. 

In [9]:
create_emotions_table = """
CREATE TABLE IF NOT EXISTS emotions (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  name TEXT NOT NULL,
  user_id INTEGER NOT NULL, 
  fragment_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (fragment_id) REFERENCES fragment (id)
);
"""

execute_query(connection, create_emotions_table)

Query executed successfully


# Вставка записей

В этом разделе мы узнаем, как вставлять записи. 

Чтобы вставить записи в базу данных SQLite, мы можем использовать ту же execute_query() функцию, что и для создания таблиц. Для этого сначала нужно сохранить в виде строки запрос INSERT INTO. Затем нужно передать объект connection и строковый запрос в execute_query(). Вставим для примера пять записей в таблицу users:

In [10]:
create_users = """
INSERT INTO
  users (name, surname, age, gender, school, class)
VALUES
  ('Иван', 'Петров', 15, 'male', 'МБОУ «СОШ №8»', 9),
  ('Елена', 'Сахарова', 12, 'female', 'МБОУ «СЭЛ № 45»', 7),
  ('Борис', 'Романенко', 13, 'male', 'МАОУ «Школа «Липовая роща»', 8),
  ('Михаил', 'Степанов', 14, 'male', 'МБОУ «СОШ №35»', 12),
  ('Елизавета', 'Куликовская', 11, 'female', 'МБОУ «Гимназия №6 им. Габдуллы Тукая»', 17);
"""

execute_query(connection, create_users) 

Query executed successfully


Поскольку мы установили автоинкремент для столбца id, нам не нужно указывать его дополнительно. Таблица users будет автоматически заполнена пятью записями со значениями id от 1 до 5.

Вставим в таблицу fragments три записи. Для наглядности фрагментами будут небольше предложения.

In [11]:
create_fragments = """
INSERT INTO
  fragments (chapter, content, hero, word, user_id)
VALUES
  (4, "Поэт бросился бежать к турникету, как только услыхал первый вопль, и видел, как голова подскакивала на мостовой.", " ", " ", 1),
  (21, "Лишь только Маргарита коснулась влажной травы, музыка под вербами ударила сильнее, и веселее взлетел сноп искр из костра.", "Маргарита", " ", 5),
  (17, "— Нету, — отвечал бухгалтер, все более бледнея и разводя руками.", " ", "рука", 3);
"""

execute_query(connection, create_fragments)

Query executed successfully


Необходимо добавить поле эмоция в таблицу фрагмент?

Важно отметить, что столбец user_id таблицы fragments является внешним ключом, который ссылается на столбец таблицы users. Это означает, что столбец user_id должен содержать значение, которое уже существует в столбце id таблицы users. Если его не существует, мы получим сообщение об ошибке.

Следующий скрипт вставляет записи в таблицы keywords и emotions:

In [12]:
create_keywords = """
INSERT INTO
  keywords (title, synonyms, word_emotion, user_id, fragment_id)
VALUES
  ('голова', "", "страх", 1, 1),
  ('костер', "огонь, пламя", "радость", 5, 2),
  ('трава', "", "радость", 4, 2),
  ('рука', "ладонь", "страх", 4, 3),
  ('бледность', "бледнеть", "страх", 2, 3),
  ('музыка', "", "радость", 3, 2);
"""

create_emotions = """
INSERT INTO
  emotions (name, user_id, fragment_id)
VALUES
  ("страх", 4, 1),
  ("радость", 2, 2),
  ("страх", 1, 3);
"""

execute_query(connection, create_keywords)
execute_query(connection, create_emotions)  

Query executed successfully
Query executed successfully


# Извлечение данных из записей

Чтобы выбрать записи в SQLite, можно снова использовать cursor.execute(). Однако после этого потребуется вызвать метод курсора fetchall(). Этот метод возвращает список кортежей, где каждый кортеж сопоставлен с соответствующей строкой в извлеченных записях. Чтобы упростить процесс, напишем функцию execute_read_query():

In [13]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

Эта функция принимает объект connection и SELECT-запрос, а возвращает выбранную запись.

1. Давайте выберем все записи из таблицы users:

In [14]:
select_users = "SELECT * from users"
users = execute_read_query(connection, select_users)

for user in users:
    print(user)

(1, 'Иван', 'Петров', 15, 'male', 'МБОУ «СОШ №8»', 9)
(2, 'Елена', 'Сахарова', 12, 'female', 'МБОУ «СЭЛ № 45»', 7)
(3, 'Борис', 'Романенко', 13, 'male', 'МАОУ «Школа «Липовая роща»', 8)
(4, 'Михаил', 'Степанов', 14, 'male', 'МБОУ «СОШ №35»', 12)
(5, 'Елизавета', 'Куликовская', 11, 'female', 'МБОУ «Гимназия №6 им. Габдуллы Тукая»', 17)


В приведенном выше скрипте запрос SELECT забирает всех пользователей из таблицы users. Результат передается в написанную нами функцию execute_read_query(), возвращающую все записи из таблицы users.

Примечание. Не рекомендуется использовать SELECT * для больших таблиц, так как это может привести к большому числу операций ввода-вывода, которые увеличивают сетевой трафик.

2. Таким же образом вы можете извлечь все записи из таблицы fragments:

In [15]:
select_fragments = "SELECT * FROM fragments"
fragments = execute_read_query(connection, select_fragments)

for fragment in fragments:
    print(fragment)

(1, 4, 'Поэт бросился бежать к турникету, как только услыхал первый вопль, и видел, как голова подскакивала на мостовой.', ' ', ' ', 1)
(2, 21, 'Лишь только Маргарита коснулась влажной травы, музыка под вербами ударила сильнее, и веселее взлетел сноп искр из костра.', 'Маргарита', ' ', 5)
(3, 17, '— Нету, — отвечал бухгалтер, все более бледнея и разводя руками.', ' ', 'рука', 3)


3. Посмотрим также на таблицу keywords

In [16]:
select_keywords = "SELECT * FROM keywords"
keywords = execute_read_query(connection, select_keywords)

for keyword in keywords:
    print(keyword)

(1, 'голова', '', 'страх', 1, 1)
(2, 'костер', 'огонь, пламя', 'радость', 5, 2)
(3, 'трава', '', 'радость', 4, 2)
(4, 'рука', 'ладонь', 'страх', 4, 3)
(5, 'бледность', 'бледнеть', 'страх', 2, 3)
(6, 'музыка', '', 'радость', 3, 2)


4. Вы также можете выполнять более сложные запросы, включающие операции типа JOIN для извлечения данных из двух связанных таблиц. Например, следующий скрипт возвращает идентификаторы и имена пользователей, а также описание фрагментов, размеченных этими пользователями:

In [17]:
select_users_fragments = """
SELECT
  users.id,
  users.name,
  fragments.content
FROM
  fragments
  INNER JOIN users ON users.id = fragments.user_id
"""

users_fragments = execute_read_query(connection, select_users_fragments)

for users_fragment in users_fragments:
    print(users_fragment)

(1, 'Иван', 'Поэт бросился бежать к турникету, как только услыхал первый вопль, и видел, как голова подскакивала на мостовой.')
(5, 'Елизавета', 'Лишь только Маргарита коснулась влажной травы, музыка под вербами ударила сильнее, и веселее взлетел сноп искр из костра.')
(3, 'Борис', '— Нету, — отвечал бухгалтер, все более бледнея и разводя руками.')


6. Следующий скрипт возвращает все фрагменты вместе с ключевыми словами и именами пользователей, которые разметили ключевые слова:

In [18]:
select_fragments_keywords_users = """
SELECT
  fragments.content as fragment,
  title as key,
  name
FROM
  fragments
  INNER JOIN keywords ON fragments.id = keywords.fragment_id
  INNER JOIN users ON users.id = keywords.user_id
"""

fragments_keywords_users = execute_read_query(
    connection, select_fragments_keywords_users)

for fragments_keywords_user in fragments_keywords_users:
    print(fragments_keywords_user)

('Поэт бросился бежать к турникету, как только услыхал первый вопль, и видел, как голова подскакивала на мостовой.', 'голова', 'Иван')
('Лишь только Маргарита коснулась влажной травы, музыка под вербами ударила сильнее, и веселее взлетел сноп искр из костра.', 'костер', 'Елизавета')
('Лишь только Маргарита коснулась влажной травы, музыка под вербами ударила сильнее, и веселее взлетел сноп искр из костра.', 'трава', 'Михаил')
('— Нету, — отвечал бухгалтер, все более бледнея и разводя руками.', 'рука', 'Михаил')
('— Нету, — отвечал бухгалтер, все более бледнея и разводя руками.', 'бледность', 'Елена')
('Лишь только Маргарита коснулась влажной травы, музыка под вербами ударила сильнее, и веселее взлетел сноп искр из костра.', 'музыка', 'Борис')


Из вывода понятно, что имена столбцов не были возвращены методом fetchall(). Чтобы вернуть имена столбцов, нужно забрать атрибут description объекта cursor. 
7. Например, следующий список возвращает все имена столбцов для вышеуказанного запроса:

In [19]:
cursor = connection.cursor()
cursor.execute(select_fragments_keywords_users)
cursor.fetchall()

column_names = [content[0] for content in cursor.description]
print(column_names)

['fragment', 'key', 'name']


Используя запрос WHERE, вы можете возвращать более конкретные результаты. 
8. Выполним SELECT-запрос, который возвращает текст фрагмента и общее количество его оценок:

In [20]:
select_fragment_emotions = """
SELECT
  content as Post,
  COUNT(emotions.id) as Emotions
FROM
  emotions,
  fragments
WHERE
  fragments.id = emotions.fragment_id
GROUP BY
  emotions.fragment_id
"""

fragment_emotions = execute_read_query(connection, select_fragment_emotions)

for fragment_emotion in fragment_emotions:
    print(fragment_emotion)

('Поэт бросился бежать к турникету, как только услыхал первый вопль, и видел, как голова подскакивала на мостовой.', 1)
('Лишь только Маргарита коснулась влажной травы, музыка под вербами ударила сильнее, и веселее взлетел сноп искр из костра.', 1)
('— Нету, — отвечал бухгалтер, все более бледнея и разводя руками.', 1)


# Обновление и удаление записей таблицы

Обновление записей в SQLite выглядит довольно просто. Снова можно применить execute_query(). В качестве примера обновим текст поста с id равным 2. Сначала создадим описание для SELECT:

In [21]:
select_fragment_content = """
SELECT content FROM fragments WHERE id = 2
"""

fragment_content = execute_read_query(connection, select_fragment_content)

for content in fragment_content:
    print(content)

('Лишь только Маргарита коснулась влажной травы, музыка под вербами ударила сильнее, и веселее взлетел сноп искр из костра.',)


Следующий скрипт обновит описание:

In [22]:
update_fragment_content = """
UPDATE
  fragments
SET
  content = "Но дальнейшее уже не интересовало Маргариту. Прицелившись, чтобы не задеть за какой-нибудь провод, она покрепче сжала щетку и в мгновение оказалась выше злополучного дома."
WHERE
  id = 2
"""

execute_query(connection, update_fragment_content)

Query executed successfully


Повторим запрос

In [23]:
select_fragment_content = """
SELECT content FROM fragments WHERE id = 2
"""

fragment_content = execute_read_query(connection, select_fragment_content)

for content in fragment_content:
    print(content)

('Но дальнейшее уже не интересовало Маргариту. Прицелившись, чтобы не задеть за какой-нибудь провод, она покрепче сжала щетку и в мгновение оказалась выше злополучного дома.',)


В качестве примера удалим ключевое слово с id равным 5:

In [24]:
delete_keyword = "DELETE FROM keywords WHERE id = 5"
execute_query(connection, delete_keyword)

Query executed successfully


Посмотрим, как выглядит таблица

In [25]:
select_keywords = "SELECT * FROM keywords"
keywords = execute_read_query(connection, select_keywords)

for keyword in keywords:
    print(keyword)

(1, 'голова', '', 'страх', 1, 1)
(2, 'костер', 'огонь, пламя', 'радость', 5, 2)
(3, 'трава', '', 'радость', 4, 2)
(4, 'рука', 'ладонь', 'страх', 4, 3)
(6, 'музыка', '', 'радость', 3, 2)


In [26]:
connection.close()